In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import json
import time
from math import ceil

In [2]:
def get_id_from_link(link):
    return re.search(r'id=.+?&', link)[0][3:-1]

In [34]:
def one_page_to_table(page_html):
    df = pd.read_html(page_html)[5]
    soup = BeautifulSoup(page_html)
    urls = [tag.get('href') for tag in soup.find('ul', {'id': 'resultsList'}).find('tbody').find_all('a')]
    try:
        df['URL'] = urls
    except:
        df = pd.read_html(page_html)[2]
        df['URL'] = urls
    df['ID'] = df['URL'].apply(get_id_from_link)
    return df

In [4]:
def get_n_cases(page_html):
    return int(BeautifulSoup(page_html).find('div', {'class':'resultCount'}).get('data-total').replace(' ', ''))

In [5]:
def get_n_pages(n_cases):
    n_steps = ceil(n_cases / 20)
    return n_steps

### Работа парсера ссылок с ГАС на примере 2018 года

#### 105 статья (случай, когда нужна статья целиком, все части)

In [25]:
browser = wb.Chrome()
article = 105
year = 2018
url = f'https://bsr.sudrf.ru/bigs/portal.html#%7B%22groups%22:%5B%22Уголовные%20дела%22%5D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20{article}%5C%22,%5C%22sQuery%5C%22:null%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22Уголовные%20дела%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D,%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22common%5C%22,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22,%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_user_document_num%5C%22,%5C%22operator%5C%22:%5C%22GE%5C%22,%5C%22query%5C%22:%5C%221%5C%22%7D%5D%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22FACET_QUERY###case_user_document_num###Да%22%7D,%7B%22type%22:%22SQ%22,%22queryId%22:%227f9e8ff8-4bc8-46aa-bcbd-f2b3ed5f159f%22,%22operator%22:%22AND%22%7D%5D%7D,%22simpleSearchFieldsBundle%22:null,%22filterGroups%22:%5B%7B%22filterQueries%22:%5B%7B%22field%22:%22case_user_doc_result_date%22,%22query%22:%22{year}%20год%22,%22not%22:false,%22_invertedNot%22:false,%22_and%22:false%7D,%7B%22field%22:%22case_user_document_type%22,%22query%22:%22Приговор%22,%22not%22:false%7D,%7B%22field%22:%22case_user_doc_result%22,%22query%22:%22Вынесен%20ПРИГОВОР%22,%22not%22:false,%22_invertedNot%22:false,%22_and%22:false%7D%5D,%22groupMode%22:%22OR%22,%22not%22:false%7D%5D%7D'
browser.get(url)
time.sleep(2)
page_html = browser.page_source
n_cases = get_n_cases(page_html)
n_pages = get_n_pages(n_cases)
print(n_cases, n_pages) # количество приговоров и страниц выдачи

5698 285


In [26]:
# при возникновении ошибки со стороны ГАС перезапустите ячейку - парсер продолжить работать с прежнего места
if 'all_data' not in locals():
    all_data = pd.DataFrame()
curr_position = len(all_data)
for i in range(curr_position, n_cases+1, 20):
    browser.get('http://www.sudrf.ru/')
    url = f'https://bsr.sudrf.ru/bigs/portal.html#%7B%22start%22:{i},%22rows%22:20,%22groups%22:%5B%22Уголовные%20дела%22%5D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20{article}%5C%22,%5C%22sQuery%5C%22:null%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22Уголовные%20дела%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D,%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22common%5C%22,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22,%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_user_document_num%5C%22,%5C%22operator%5C%22:%5C%22GE%5C%22,%5C%22query%5C%22:%5C%221%5C%22%7D%5D%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22FACET_QUERY###case_user_document_num###Да%22%7D,%7B%22type%22:%22SQ%22,%22queryId%22:%227f9e8ff8-4bc8-46aa-bcbd-f2b3ed5f159f%22,%22operator%22:%22AND%22%7D%5D%7D,%22simpleSearchFieldsBundle%22:null,%22filterGroups%22:%5B%7B%22filterQueries%22:%5B%7B%22field%22:%22case_user_doc_result_date%22,%22query%22:%22{year}%20год%22,%22not%22:false,%22_invertedNot%22:false,%22_and%22:false%7D,%7B%22field%22:%22case_user_document_type%22,%22query%22:%22Приговор%22,%22not%22:false%7D,%7B%22field%22:%22case_user_doc_result%22,%22query%22:%22Вынесен%20ПРИГОВОР%22,%22not%22:false,%22_invertedNot%22:false,%22_and%22:false%7D%5D,%22groupMode%22:%22OR%22,%22not%22:false%7D%5D%7D'
    browser.get(url)

    element = WebDriverWait(browser, 25).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#resultViewSelectorOuter > a:nth-child(2)"))
    )
    
    while 'resultHeader' not in browser.page_source:
        time.sleep(0.1)
    time.sleep(5)
    
    browser.find_element_by_css_selector('#resultViewSelectorOuter > a:nth-child(2)').click()

    while 'id="resultsList"' not in browser.page_source:
        time.sleep(0.1)
    time.sleep(1)
    data_per_step = one_page_to_table(browser.page_source)
    all_data = pd.concat([all_data, data_per_step])
    if len(all_data)%100==0:
        print(f'{len(all_data)}/{n_cases}')
        all_data.to_excel(f'{article}_{year}_cases_links.xlsx')
all_data

100/5698


C:\Users\svetl\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'https://bsr.sudrf.ru/bigs/showDocument.html' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))


200/5698
300/5698
400/5698
500/5698
600/5698
700/5698
800/5698
900/5698
1000/5698
1100/5698
1200/5698
1300/5698
1400/5698
1500/5698
1600/5698
1700/5698
1800/5698
1900/5698
2000/5698
2100/5698
2200/5698
2300/5698
2400/5698
2500/5698
2600/5698
2700/5698
2800/5698
2900/5698
3000/5698
3100/5698
3200/5698
3300/5698
3400/5698
3500/5698
3600/5698
3700/5698
3800/5698
3900/5698
4000/5698
4100/5698
4200/5698
4300/5698
4400/5698
4500/5698
4600/5698
4700/5698
4800/5698
4900/5698
5000/5698
5100/5698
5200/5698
5300/5698
5400/5698
5500/5698
5600/5698


,№,Аннотация,Номер дела (материала),Тип документа,Статья УК РФ,Дата поступления,Дата решения,Субъект РФ,Наименование суда,ФИО,Результат,URL,ID
0,1,Приговор,2-31/2018,Приговор,Статья 105 Часть 2 п.г; Статья 105 Часть 2 п.г,13.09.2018,22.10.2018,Республика Татарстан,Верховный Суд Республики Татарстан,Парахин А.А.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,ec89b941970471362dc925f201c91098
1,2,Приговор,1-63/2018 (1-1539/2017;),Приговор,Статья 105 Часть 2 п.ж; Статья 105 Часть 2 п.п...,01.12.2017,01.06.2018,Республика Татарстан,Набережночелнинский городской суд,Хуббатуллин Н.Б.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,1b61023f0dc291d773ca92fe22f85f7d
2,3,Приговор,2-4/2018,Приговор,Статья 105 Часть 2 п.д,29.01.2018,29.03.2018,Тверская область,Тверской областной суд,Тертерян А.Н.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,d6bc8a8bb14a4db4ce7d50ba8073b9fa
3,4,Приговор,1-8/2018 (1-49/2017;),Приговор,Статья 105 Часть 2 п.в,03.11.2017,30.01.2018,Свердловская область,Свердловский областной суд,Поличинский А.А.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,3b8b13f92c16de8465ace460479d93ef
4,5,Приговор,2-3/2018,Приговор,Статья 105 Часть 2 п.ж,19.01.2018,06.03.2018,Чувашская Республика - Чувашия,Верховный Суд Чувашской Республики,Сидорова М.А.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,080ea8dbc5586681f6308698c13ab858
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,5694,Приговор,1-55/2018,Приговор,Статья 166 Часть 1; Статья 158 Часть 1; Статья...,18.06.2018,18.07.2018,Пензенская область,Нижнеломовский районный суд,Зуев Денис Олегович,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,253dc06ad9e72adfb72fb0fef8d58f5e
14,5695,Приговор,1-23/2018,Приговор,Статья 167 Часть 2; Статья 139 Часть 1; Статья...,30.03.2018,26.04.2018,Архангельская область,Пинежский районный суд,Шехурин Николай Александрович,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,e61cb29bb1224ae8bab51ac7b7c3099b
15,5696,Приговор,1-223/2018,Приговор,Статья 111 Часть 2 п.з; Статья 112 Часть 2 п.з...,03.09.2018,13.11.2018,Красноярский край,Железногорский городской суд,Евсеев Андрей Анатольевич,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,4c8a45287e2c1753de6ff91831c4757e
16,5697,Приговор,1-274/2018,Приговор,"Статья 30 Часть 3, Статья 167 Часть 2; Статья ...",05.10.2018,17.12.2018,Архангельская область,Котласский городской суд,Яковлева Екатерина Алексеевна,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,95a2d6306e5644e0614837ef21ea7540


In [3]:
all_data.to_excel(f'{article}_{year}_cases_links.xlsx')

In [ ]:
del all_data

#### 111 часть 4 (случай, когда нужна только часть статьи)

In [28]:
browser = wb.Chrome()
year = 2018
complex_article = '111_4'
article = complex_article.split('_')[0]
part = complex_article.split('_')[1]
url = f'https://bsr.sudrf.ru/bigs/portal.html#%7B%22groups%22:%5B%22Уголовные%20дела%22%5D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20{article}%20%20Часть%20{part}%5C%22,%5C%22sQuery%5C%22:null%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22Уголовные%20дела%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D,%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22common%5C%22,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22,%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_user_document_num%5C%22,%5C%22operator%5C%22:%5C%22GE%5C%22,%5C%22query%5C%22:%5C%221%5C%22%7D%5D%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22FACET_QUERY###case_user_document_num###Да%22%7D,%7B%22type%22:%22SQ%22,%22queryId%22:%227f9e8ff8-4bc8-46aa-bcbd-f2b3ed5f159f%22,%22operator%22:%22AND%22%7D%5D%7D,%22simpleSearchFieldsBundle%22:null,%22filterGroups%22:%5B%7B%22filterQueries%22:%5B%7B%22field%22:%22case_user_doc_result_date%22,%22query%22:%22{year}%20год%22,%22not%22:false,%22_invertedNot%22:false,%22_and%22:false%7D,%7B%22field%22:%22case_user_document_type%22,%22query%22:%22Приговор%22,%22not%22:false,%22_invertedNot%22:false,%22_and%22:false%7D,%7B%22field%22:%22case_user_doc_result%22,%22query%22:%22Вынесен%20ПРИГОВОР%22,%22not%22:false,%22_invertedNot%22:false,%22_and%22:false%7D%5D,%22groupMode%22:%22OR%22,%22not%22:false%7D%5D,%22noOrpho%22:false%7D'
browser.get(url)
time.sleep(2)
page_html = browser.page_source
n_cases = get_n_cases(page_html)
n_pages = get_n_pages(n_cases)
print(n_cases, n_pages)

3175 159


In [29]:
# при возникновении ошибки со стороны ГАС перезапустите ячейку - парсер продолжить работать с прежнего места
if 'all_data' not in locals():
    all_data = pd.DataFrame()
curr_position = len(all_data)
for i in range(curr_position, n_cases+1, 20):
    browser.get('http://www.sudrf.ru/')
    url = f'https://bsr.sudrf.ru/bigs/portal.html#%7B%22start%22:{i},%22rows%22:20,%22groups%22:%5B%22Уголовные%20дела%22%5D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20{article}%20%20Часть%20{part}%5C%22,%5C%22sQuery%5C%22:null%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22Уголовные%20дела%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D,%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22common%5C%22,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22,%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_user_document_num%5C%22,%5C%22operator%5C%22:%5C%22GE%5C%22,%5C%22query%5C%22:%5C%221%5C%22%7D%5D%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22FACET_QUERY###case_user_document_num###Да%22%7D,%7B%22type%22:%22SQ%22,%22queryId%22:%227f9e8ff8-4bc8-46aa-bcbd-f2b3ed5f159f%22,%22operator%22:%22AND%22%7D%5D%7D,%22simpleSearchFieldsBundle%22:null,%22filterGroups%22:%5B%7B%22filterQueries%22:%5B%7B%22field%22:%22case_user_doc_result_date%22,%22query%22:%22{year}%20год%22,%22not%22:false,%22_invertedNot%22:false,%22_and%22:false%7D,%7B%22field%22:%22case_user_document_type%22,%22query%22:%22Приговор%22,%22not%22:false,%22_invertedNot%22:false,%22_and%22:false%7D,%7B%22field%22:%22case_user_doc_result%22,%22query%22:%22Вынесен%20ПРИГОВОР%22,%22not%22:false,%22_invertedNot%22:false,%22_and%22:false%7D%5D,%22groupMode%22:%22OR%22,%22not%22:false%7D%5D,%22noOrpho%22:false%7D'
    browser.get(url)

    element = WebDriverWait(browser, 25).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#resultViewSelectorOuter > a:nth-child(2)"))
    )
    
    while 'resultHeader' not in browser.page_source:
        time.sleep(0.1)
    time.sleep(5)
    
    browser.find_element_by_css_selector('#resultViewSelectorOuter > a:nth-child(2)').click()

    while 'id="resultsList"' not in browser.page_source:
        time.sleep(0.1)
    time.sleep(1)
    data_per_step = one_page_to_table(browser.page_source)
    all_data = pd.concat([all_data, data_per_step])
    if len(all_data)%100==0:
        print(f'{len(all_data)}/{n_cases}')
        all_data.to_excel(f'{complex_article}_{year}_cases_links.xlsx')
all_data

100/3175
200/3175
300/3175
400/3175
500/3175
600/3175
700/3175
800/3175
900/3175
1000/3175
1100/3175
1200/3175
1300/3175
1400/3175
1500/3175
1600/3175
1700/3175
1800/3175
1900/3175
2000/3175
2100/3175
2200/3175
2300/3175
2400/3175
2500/3175
2600/3175
2700/3175
2800/3175
2900/3175
3000/3175
3100/3175


,№,Аннотация,Номер дела (материала),Тип документа,Статья УК РФ,Дата поступления,Дата решения,Субъект РФ,Наименование суда,ФИО,Результат,URL,ID
0,1,Статья 111 Часть 4; Статья 111 Часть 4...Приго...,1-587/2018,Приговор,Статья 111 Часть 4; Статья 111 Часть 4,24.08.2018,27.12.2018,Московская область,Орехово-Зуевский городской суд,Лобузов А.А.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,8ca3c0e82417771b7327e0fb748cdda6
1,2,Статья 111 Часть 4; Статья 111 Часть 4...Приго...,1-196/2018,Приговор,Статья 111 Часть 4; Статья 111 Часть 4,24.07.2018,09.10.2018,Курганская область,Шадринский районный суд,Сумин Ю.В.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,f43b786f0f54aab8983531e471961966
2,3,Статья 111 Часть 4; Статья 111 Часть 4...Приго...,1-74/2018 (1-1168/2017;),Приговоры,Статья 111 Часть 4; Статья 111 Часть 4,27.12.2017,26.01.2018,Вологодская область,Вологодский городской суд,Архипов А.А.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,020e244c83624b727a9060daaaa14048
3,4,Статья 111 Часть 4; Статья 111 Часть 4...Приго...,1-189/2018,Приговор,Статья 111 Часть 4; Статья 111 Часть 4,11.07.2018,20.08.2018,Хабаровский край,Суд района имени Лазо,Не заполнено,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,a026c4b8eac7a92bd6a5d5d510b57ed2
4,5,Статья 111 Часть 4; Статья 111 Часть 4...Приго...,1-78/2018,Приговор,Статья 111 Часть 4; Статья 111 Часть 4,18.09.2018,11.12.2018,Смоленская область,Духовщинский районный суд,Москвичев Виктор Сергеевич,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,6863c4bb5fedb8416299e2e130f3cfa1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,3171,Приговоры,1-175/2018,Приговоры,"Статья 30 Часть 3, Статья 158 Часть 2 п.б; Ста...",05.02.2018,23.04.2018,Вологодская область,Вологодский городской суд,Мартынов М.А.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,3d286e017fa534c95d76d5db970553c0
11,3172,Приговор,1-164/2018,Приговор,Статья 115 Часть 1; Статья 167 Часть 2; Статья...,14.03.2018,12.10.2018,Республика Дагестан,Кировский районный суд г. Махачкалы,Мусаев С.Р.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,f26617f8a42983aba6b49f429ab0d0a2
12,3173,Приговор,1-329/2018,Приговор,Статья 306 Часть 2; Статья 166 Часть 2 п.а; Ст...,23.05.2018,18.07.2018,Республика Татарстан,Нижнекамский городской суд,"Хисамутдинов Р.Д., Аминов С.Ю.",Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,1671a2756aeaeb5ff6c279d8db79fa9e
13,3174,Приговор,2-22/2018,Приговор,Статья 111 Часть 4; Статья 159 Часть 4; Статья...,27.03.2018,03.05.2018,Нижегородская область,Нижегородский областной суд,Велькин Д.В.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,ec3ffeb4f18acde40b33a75e9454a2da


In [4]:
all_data.to_excel(f'111_4_{year}_cases_links.xlsx')

In [ ]:
del all_data

#### 107 статья (случай, когда нужна статья целиком, все части)

In [31]:
browser = wb.Chrome()
article = 107
year = 2018
url = f'https://bsr.sudrf.ru/bigs/portal.html#%7B%22groups%22:%5B%22Уголовные%20дела%22%5D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20{article}%5C%22,%5C%22sQuery%5C%22:null%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22Уголовные%20дела%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D,%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22common%5C%22,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22,%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_user_document_num%5C%22,%5C%22operator%5C%22:%5C%22GE%5C%22,%5C%22query%5C%22:%5C%221%5C%22%7D%5D%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22FACET_QUERY###case_user_document_num###Да%22%7D,%7B%22type%22:%22SQ%22,%22queryId%22:%227f9e8ff8-4bc8-46aa-bcbd-f2b3ed5f159f%22,%22operator%22:%22AND%22%7D%5D%7D,%22simpleSearchFieldsBundle%22:null,%22filterGroups%22:%5B%7B%22filterQueries%22:%5B%7B%22field%22:%22case_user_doc_result_date%22,%22query%22:%22{year}%20год%22,%22not%22:false,%22_invertedNot%22:false,%22_and%22:false%7D,%7B%22field%22:%22case_user_document_type%22,%22query%22:%22Приговор%22,%22not%22:false%7D,%7B%22field%22:%22case_user_doc_result%22,%22query%22:%22Вынесен%20ПРИГОВОР%22,%22not%22:false,%22_invertedNot%22:false,%22_and%22:false%7D%5D,%22groupMode%22:%22OR%22,%22not%22:false%7D%5D%7D'
browser.get(url)
# element = WebDriverWait(browser, 30).until(
#     EC.presence_of_element_located((By.CSS_SELECTOR, "#resultViewSelectorOuter > a:nth-child(2)"))
# )
time.sleep(2)
page_html = browser.page_source
n_cases = get_n_cases(page_html)
n_pages = get_n_pages(n_cases)
print(n_cases, n_pages)

45 3


In [32]:
# при возникновении ошибки со стороны ГАС перезапустите ячейку - парсер продолжить работать с прежнего места
if 'all_data' not in locals():
    all_data = pd.DataFrame()
curr_position = len(all_data)
for i in range(curr_position, n_cases+1, 20):
    browser.get('http://www.sudrf.ru/')
    url = f'https://bsr.sudrf.ru/bigs/portal.html#%7B%22start%22:{i},%22rows%22:20,%22groups%22:%5B%22Уголовные%20дела%22%5D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20{article}%5C%22,%5C%22sQuery%5C%22:null%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22Уголовные%20дела%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D,%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22common%5C%22,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22,%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_user_document_num%5C%22,%5C%22operator%5C%22:%5C%22GE%5C%22,%5C%22query%5C%22:%5C%221%5C%22%7D%5D%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22FACET_QUERY###case_user_document_num###Да%22%7D,%7B%22type%22:%22SQ%22,%22queryId%22:%227f9e8ff8-4bc8-46aa-bcbd-f2b3ed5f159f%22,%22operator%22:%22AND%22%7D%5D%7D,%22simpleSearchFieldsBundle%22:null,%22filterGroups%22:%5B%7B%22filterQueries%22:%5B%7B%22field%22:%22case_user_doc_result_date%22,%22query%22:%22{year}%20год%22,%22not%22:false,%22_invertedNot%22:false,%22_and%22:false%7D,%7B%22field%22:%22case_user_document_type%22,%22query%22:%22Приговор%22,%22not%22:false%7D,%7B%22field%22:%22case_user_doc_result%22,%22query%22:%22Вынесен%20ПРИГОВОР%22,%22not%22:false,%22_invertedNot%22:false,%22_and%22:false%7D%5D,%22groupMode%22:%22OR%22,%22not%22:false%7D%5D%7D'
    browser.get(url)

    element = WebDriverWait(browser, 25).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#resultViewSelectorOuter > a:nth-child(2)"))
    )
    
    while 'resultHeader' not in browser.page_source:
        time.sleep(0.1)
    time.sleep(5)
    
    browser.find_element_by_css_selector('#resultViewSelectorOuter > a:nth-child(2)').click()

    while 'id="resultsList"' not in browser.page_source:
        time.sleep(0.1)
    time.sleep(1)
    data_per_step = one_page_to_table(browser.page_source)
    all_data = pd.concat([all_data, data_per_step])
    if len(all_data)%100==0:
        print(f'{len(all_data)}/{n_cases}')
        all_data.to_excel(f'{article}_{year}_cases_links.xlsx')
all_data

,№,Аннотация,Номер дела (материала),Тип документа,Статья УК РФ,Дата поступления,Дата решения,Субъект РФ,Наименование суда,ФИО,Результат,URL,ID
0,1,Статья 107 Часть 1...Приговор...Вынесен ПРИГОВОР,1-34/2018,Приговор,Статья 107 Часть 1,12.01.2018,23.01.2018,Московская область,Ступинский городской суд,Лутчин В.В.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,27a1fd89fa9f5cae4f17863da6ab05dd
1,2,Статья 107 Часть 1...Приговор...Вынесен ПРИГОВОР,1-285/2018,Приговор,Статья 107 Часть 1,28.09.2018,15.10.2018,Оренбургская область,Советский районный суд г. Орска,Алиханов С.Т.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,0ad7d7cbcd90e9b20c6ea81d917d5d05
2,3,Статья 107 Часть 1...Приговор...Вынесен ПРИГОВОР,1-252/2018,Приговор,Статья 107 Часть 1,22.02.2018,07.03.2018,Ростовская область,Новочеркасский городской суд,Пятибратова Е.В.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,939c7ee331f5aa6dd05adaec33441141
3,4,Статья 107 Часть 1...Приговор...Вынесен ПРИГОВОР,1-76/2018,Приговор,Статья 107 Часть 1,06.02.2018,27.02.2018,Ростовская область,Красносулинский районный суд,Першина Е.С.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,70b12576b300062582de1abe769f89a6
4,5,Статья 107 Часть 1...Приговор...Вынесен ПРИГОВОР,1-346/2018,Приговор,Статья 107 Часть 1,15.05.2018,20.06.2018,Оренбургская область,Ленинский районный суд г. Оренбурга,Носырева О.Г.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,8ff2202ce92636afeb68011e52779f9c
5,6,Статья 107 Часть 1...Приговор...Вынесен ПРИГОВОР,1-98/2018,Приговор,Статья 107 Часть 1,09.01.2018,01.02.2018,Челябинская область,Металлургический районный суд г. Челябинска,Аетбаев Д.Р.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,4bb80aa68340e9f7c059fd88d635b0d3
6,7,Статья 107 Часть 1...Приговор...Вынесен ПРИГОВОР,1-66/2018,Приговор,Статья 107 Часть 1,25.01.2018,07.02.2018,Московская область,Дмитровский городской суд,Васинова П.Н.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,94783011c636b12c4da9e8f053eeb988
7,8,Статья 107 Часть 1...Приговор...Вынесен ПРИГОВОР,1-82/2018,Приговор,Статья 107 Часть 1,30.01.2018,16.02.2018,Московская область,Дмитровский городской суд,Карпов В.А.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,30e44682c600244487f040488374eef4
8,9,Статья 107 Часть 1...Приговор...Вынесен ПРИГОВОР,1-78/2018 (1-665/2017;),Приговор,Статья 107 Часть 1,18.12.2017,24.01.2018,Московская область,Сергиево-Посадский городской суд,Дворцевой А.А.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,183e46215990f94e09435643b9a619e7
9,10,Статья 107 Часть 1...Приговор...Вынесен ПРИГОВОР,1-67/2018,Приговор,Статья 107 Часть 1,31.05.2018,26.06.2018,Московская область,Дубненский городской суд,Архипов С.В.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,b73a10c1f61e58732f6cb7ddfae7303e


In [1]:
all_data.to_excel(f'{article}_{year}_cases_links.xlsx')

In [ ]:
del all_data